In [46]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence


from collections import Counter
from sklearn.metrics import accuracy_score
import operator
from tqdm import tqdm
import multiprocessing

In [47]:
NUMBER_OF_CORES = 16

### Method for getting k-skip-n-grams from the sequence

In [48]:
# k - words between, if k = 0 then we get just n-grams
# n - number of words in tuple, n = 2 -> bigram
# also all k_skip_n_grams are sorted inside the tuple in order to get rid of duplicates
def get_k_skip_n_grams(review, k_value=0, n_value=1, unique=False):
    unsorted_k_skip_n_grams = zip(*[review[i*(k_value+1):] for i in range(n_value)])
    sorted_k_skip_n_grams = [tuple(sorted(k_skip_n_gram)) for k_skip_n_gram in unsorted_k_skip_n_grams]
    if unique:
        return set(sorted_k_skip_n_grams)
    else:
        return sorted_k_skip_n_grams

### Method for creating dictionary of k_skip_n_grams

In [49]:
import functools
import itertools
import math
TH = 0


# returns a Counter object: 
# key - k_skip_n_gram
# values - counter of k_skip_n_gram
def get_counter_for_k_skip_n_grams(reviews, k_value=0, n_value=2, unique=False):
    k_skip_n_grams_lists = POOL.map(functools.partial(get_k_skip_n_grams, k_value=k_value, n_value=n_value, unique=unique), reviews)
    all_k_skip_n_grams = Counter(itertools.chain(*k_skip_n_grams_lists))
    return all_k_skip_n_grams

def get_prediction(inp, self):
        index, review = inp
        k_skip_n_grams = get_k_skip_n_grams(review, k_value=self.k, n_value=self.n)


        positive_probability = math.log10(functools.reduce(operator.mul, 
                                                map(lambda k_skip_n_gram: self.positive[k_skip_n_gram] if self.positive[k_skip_n_gram] > 0 else 1, k_skip_n_grams),
                                                1))
        negative_probability = math.log10(functools.reduce(operator.mul, 
                                                map(lambda k_skip_n_gram: self.negative[k_skip_n_gram] if self.negative[k_skip_n_gram] > 0 else 1, k_skip_n_grams),
                                                1))
        if positive_probability/self._positive_len - TH >= negative_probability/self._negative_len:
            return (index, 1)
        else:
            return (index, 0)

def get_proba_prediction(inp, self):
    index, review = inp
    k_skip_n_grams = get_k_skip_n_grams(review, k_value=self.k, n_value=self.n)

    positive_probability = math.log10(functools.reduce(operator.mul, 
                                                map(lambda k_skip_n_gram: self.positive[k_skip_n_gram] if self.positive[k_skip_n_gram] > 0 else 1, k_skip_n_grams),
                                                1))
    negative_probability = math.log10(functools.reduce(operator.mul, 
                                                map(lambda k_skip_n_gram: self.negative[k_skip_n_gram] if self.negative[k_skip_n_gram] > 0 else 1, k_skip_n_grams),
                                                1))
    return (index, positive_probability/self._positive_len, negative_probability/self._negative_len)

In [50]:
class K_skip_n_gram_probability_model():
    def __init__(self, n_value=2, k_value=1):
        self.k = k_value
        self.n = n_value
        self.positive = Counter()
        self.negative = Counter()
        self._positive_reviews = []
        self._negative_reviews = []
        self._positive_len = 0
        self._negative_len = 0
    
    # self optimisation of the model
    def optimise(self, x_train, y_train, x_test, y_test):
        k_values = [0, 1, 2, 3, 4, 5, 6]
        n_values = [1, 2, 3]
        
        # all possible cases of pair (n, k)
        cases = [(x, y) for x in n_values for y in k_values]
        print(cases)
        n = self.n
        k = self.k
        # list of tuples:
        # tuple[0]: case
        # tuple[1]: accuracy_train
        # tuple[2]: accuracy_test
        result = [] 
        
        # for all values n, k fit the model and get accuracy
        for case in cases:
            # case[0] - n
            # case[1] - k
            self.n = int(case[0])
            self.k = int(case[1])
            self.internal_fit()
            train_accuracy = accuracy_score(y_train, self.predict(x_train))
            test_accuracy = accuracy_score(y_test, self.predict(x_test))
            result.append((case, train_accuracy, test_accuracy))
            print("For case: k =", case[1], "n =", case[0], "train_acc =", train_accuracy, " test acc =", test_accuracy)
            
        
        
    def predict(self, reviews):
        result = map(functools.partial(get_prediction, self=self), enumerate(reviews))
        result = sorted(result, key=lambda x: x[0], reverse=False)
        return np.array(list(map(lambda x: x[1], result)))
            
    def predict_proba(self, reviews):
        result = map(functools.partial(get_proba_prediction, self=self), enumerate(reviews))
        result = sorted(result, key=lambda x: x[0], reverse=False)
        return np.array(list(map(lambda x: (x[1], x[2]), result)))
    
    # reviews is also used to initialise/update dictionary of n_grams
    def fit(self, reviews, labels):
        self._positive_reviews = reviews[labels == 1]
        self._negative_reviews = reviews[labels == 0]
        self.positive = get_counter_for_k_skip_n_grams(self._positive_reviews, k_value=self.k, n_value=self.n)
        self.negative = get_counter_for_k_skip_n_grams(self._negative_reviews, k_value=self.k, n_value=self.n)
        self._positive_len = float(sum(self.positive.values()))
        self._negative_len = float(sum(self.negative.values()))
    
    def internal_fit(self):
        self.positive = get_counter_for_k_skip_n_grams(self._positive_reviews, k_value=self.k, n_value=self.n)
        self.negative = get_counter_for_k_skip_n_grams(self._negative_reviews, k_value=self.k, n_value=self.n)
        self._positive_len = float(sum(self.positive.values()))
        self._negative_len = float(sum(self.negative.values()))

### Let's check our model
Use k_value = 0 and n_value = 2 (ordinary bigrams)

In [87]:
if 'POOL' in globals():
    POOL.close()
POOL = multiprocessing.Pool(NUMBER_OF_CORES)

In [52]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
max_review_length = 150
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [53]:
model = K_skip_n_gram_probability_model(n_value=3, k_value=1)

In [54]:
model.fit(X_train, y_train)

In [55]:
res = model.predict(X_test)

In [56]:
print(accuracy_score(y_train, model.predict(X_train)))
print(accuracy_score(y_test, res))

0.97596
0.75768


Here we get good accuracy on training and but not so for for the test one <br>

### Let's try to get more!

I wrote a method that tries to fit the model and to compute accuracy for different values of *n* and *k*

In [57]:
model.optimise(X_train, y_train, X_test, y_test)

[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6)]
For case: k = 0 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 1 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 2 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 3 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 4 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 5 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 6 n = 1 train_acc = 0.83068  test acc = 0.80288
For case: k = 0 n = 2 train_acc = 0.96056  test acc = 0.85624
For case: k = 1 n = 2 train_acc = 0.96544  test acc = 0.83216
For case: k = 2 n = 2 train_acc = 0.96512  test acc = 0.80872
For case: k = 3 n = 2 train_acc = 0.9636  test acc = 0.79052
For case: k = 4 n = 2 train_acc = 0.96192  test acc = 0.78256
For case: k = 5 n = 2 train_acc = 0.96264  test acc = 0.77808
For case: k = 6 n = 2 trai

We can see that the best result for training data 0.98232 (n = 3, k = 0) and it corresponds to 0.8458 for the test data <br>
But the best result for test data is 0.85624 (n = 2, k = 0). <br>
Also there is one more interesting case (n = 2, k = 1), accuracy for training data is 0.96544 and 0.83216 for the test. This result is close to (n = 3, k = 0) but here we use 1-skip-2-gram. May be it is a good idea to combine these 2 kind of features? (but not now, firtsly let's try to remove num_words limit while loading imdb data)

### Let's try to remove num_words
May be we use very few of them?

In [58]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()
max_review_length = 150
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [59]:
model.fit(X_train, y_train)
res = model.predict(X_test)

print(accuracy_score(y_train, model.predict(X_train)))
print(accuracy_score(y_test, res))

0.95476
0.62396


Hmm, not so good. <br>
Perform the same trick as before

In [60]:
model.optimise(X_train, y_train, X_test, y_test)

[(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6)]
For case: k = 0 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 1 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 2 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 3 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 4 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 5 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 6 n = 1 train_acc = 0.8904  test acc = 0.8064
For case: k = 0 n = 2 train_acc = 0.97416  test acc = 0.86204
For case: k = 1 n = 2 train_acc = 0.97596  test acc = 0.83564
For case: k = 2 n = 2 train_acc = 0.97376  test acc = 0.80588
For case: k = 3 n = 2 train_acc = 0.97328  test acc = 0.78788
For case: k = 4 n = 2 train_acc = 0.97124  test acc = 0.77688
For case: k = 5 n = 2 train_acc = 0.97184  test acc = 0.7698
For case: k = 6 n = 2 train_acc = 0.9708

We see that there is no (n, k) with result for training data more than 0.98232. It seems that not very popular words add some noise to our data.

### Let's try to combine (n = 3, k = 0 with n = 2, k = 1)
At the very beginning analyse do the result that we get using one case (n = 3, k = 0) differs from (n=2, k=1). If not then there is no sense to combine them

#### Check n=3, k =0

In [61]:
model_k_0_n_3 = K_skip_n_gram_probability_model(n_value=3, k_value=0)
model_k_0_n_3.fit(X_train, y_train)

In [62]:
proba_k_0_n_3 = model_k_0_n_3.predict_proba(X_train)
pred_k_0_n_3 = model_k_0_n_3.predict(X_train)

In [63]:
print(accuracy_score(y_train, pred_k_0_n_3))

0.9808


#### Check n=2, k =1

In [64]:
model_k_1_n_2 = K_skip_n_gram_probability_model(n_value=2, k_value=1)
model_k_1_n_2.fit(X_train, y_train)

In [65]:
proba_k_1_n_2 = model_k_1_n_2.predict_proba(X_train)
pred_k_1_n_2 = model_k_1_n_2.predict(X_train)

In [66]:
print(accuracy_score(y_train, pred_k_1_n_2))

0.97596


#### How much differents

In [67]:
differs = abs(pred_k_0_n_3 - pred_k_1_n_2)
print("Differs", sum(differs), "out of", len(pred_k_0_n_3))

Differs 457 out of 25000


We see that it differs in about 2.54% cases <br>

In [68]:
differs_from_correct_k_0_n_3 = abs(pred_k_0_n_3 - y_train)
differs_from_correct_k_1_n_2 = abs(pred_k_1_n_2 - y_train)

print("k=0, n=3:", sum(differs_from_correct_k_0_n_3), "out of 25000\nk=1, n=2:", sum(differs_from_correct_k_1_n_2), "out of 25000")

k=0, n=3: 480 out of 25000
k=1, n=2: 601 out of 25000


In [69]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_train, pred_k_0_n_3).ravel()
print("For k = 0, n = 3")
print("TN = {}, FP = {}, FN = {}, TP = {}".format(tn, fp, fn, tp))

tn, fp, fn, tp = confusion_matrix(y_train, pred_k_1_n_2).ravel()
print("For k = 1, n = 2")
print("TN = {}, FP = {}, FN = {}, TP = {}".format(tn, fp, fn, tp))

For k = 0, n = 3
TN = 12106, FP = 394, FN = 86, TP = 12414
For k = 1, n = 2
TN = 12085, FP = 415, FN = 186, TP = 12314


In [70]:
print(proba_k_0_n_3[0]*10000)
proba_k_0_n_3[differs_from_correct_k_0_n_3 == 1]*10000

[ 0.41057068  0.35839217]


array([[ 3.0590531 ,  3.04103   ],
       [ 3.73373458,  3.71342058],
       [ 0.76666234,  0.76861977],
       [ 2.75635218,  2.74798172],
       [ 3.05455717,  3.02927125],
       [ 1.61980208,  1.61946917],
       [ 0.31621605,  0.31674948],
       [ 0.38754928,  0.39404225],
       [ 2.98942058,  2.975877  ],
       [ 3.14943938,  3.13018762],
       [ 2.66365471,  2.65665944],
       [ 3.17378825,  3.17228542],
       [ 1.19691187,  1.19293327],
       [ 2.40962981,  2.40793736],
       [ 0.46165212,  0.52007053],
       [ 2.04593629,  2.07522349],
       [ 3.05452479,  3.0523358 ],
       [ 3.12990524,  3.11663693],
       [ 2.4473232 ,  2.41795186],
       [ 1.5109398 ,  1.48862741],
       [ 0.21698496,  0.2188403 ],
       [ 3.1558286 ,  3.13190802],
       [ 2.86043501,  2.86037296],
       [ 2.48072072,  2.45697993],
       [ 2.94170211,  2.89968909],
       [ 3.24011942,  3.23334443],
       [ 2.42784392,  2.37735821],
       [ 0.33587618,  0.38743197],
       [ 1.98657573,

In [71]:
print(proba_k_1_n_2[0]*10000)
proba_k_1_n_2[differs_from_correct_k_1_n_2 == 1]*10000

[ 1.13625166  1.08943289]


array([[ 1.04110569,  1.02916049],
       [ 3.80738262,  3.78849089],
       [ 1.21005774,  1.21382827],
       ..., 
       [ 2.89301008,  2.89666951],
       [ 1.10998774,  1.11748916],
       [ 1.64232461,  1.65174135]])

### Let's create a model that encapsulate 2 models with different k and n values

In [72]:
class BiModel():
    def __init__(self, k1, n1, k2, n2):
        self.model1 = K_skip_n_gram_probability_model(k_value=k1, n_value=n1)
        self.model2 = K_skip_n_gram_probability_model(k_value=k2, n_value=n2)
    def predict(self, reviews):
        proba_result = self.predict_proba(reviews)
        pred_result = map(lambda x: 1 if x[0] > x[1] else 0, proba_result)
        return np.array(list(pred_result))
            
    def predict_proba(self, reviews):
        result1 = map(functools.partial(get_proba_prediction, self=self.model1), enumerate(reviews))
        result1 = sorted(result1, key=lambda x: x[0], reverse=False)
        
        result2 = map(functools.partial(get_proba_prediction, self=self.model2), enumerate(reviews))
        result2 = sorted(result2, key=lambda x: x[0], reverse=False)
        
        return np.array(list(map(lambda x: (x[1], x[2]), result1))) + np.array(list(map(lambda x: (x[1], x[2]), result2)))
    
    def fit(self, reviews, labels):
        self.model1.fit(reviews, labels)
        self.model2.fit(reviews, labels)
        

In [73]:
bimodel1 = BiModel(k1=1, n1=2, k2=0, n2=3)

In [74]:
bimodel1.fit(X_train, y_train)
print("Train data result:", accuracy_score(y_train, bimodel1.predict(X_train)))
print("Test data result:", accuracy_score(y_test, bimodel1.predict(X_test)))

Train data result: 0.9816
Test data result: 0.85492


In [75]:
bimodel2 = BiModel(k1=1, n1=2, k2=0, n2=2)

In [76]:
bimodel2.fit(X_train, y_train)
print("Train data result:", accuracy_score(y_train, bimodel2.predict(X_train)))
print("Test data result:", accuracy_score(y_test, bimodel2.predict(X_test)))

Train data result: 0.9784
Test data result: 0.85984


#### Let's try to remove 100 the most popular words, may be result will become better?

In [77]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
max_review_length = 150
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

X_train = np.array([list(filter(lambda x: x > 100, review)) for review in X_train])
X_test = np.array([list(filter(lambda x: x > 100, review)) for review in X_test])

In [78]:
bimodel1 = BiModel(k1=1, n1=2, k2=0, n2=3)
bimodel1.fit(X_train, y_train)
print("Train data result:", accuracy_score(y_train, bimodel1.predict(X_train)))
print("Test data result:", accuracy_score(y_test, bimodel1.predict(X_test)))

Train data result: 0.99372
Test data result: 0.75916


In [79]:
bimodel2 = BiModel(k1=1, n1=2, k2=0, n2=2)
bimodel2.fit(X_train, y_train)
print("Train data result:", accuracy_score(y_train, bimodel2.predict(X_train)))
print("Test data result:", accuracy_score(y_test, bimodel2.predict(X_test)))

Train data result: 0.99664
Test data result: 0.82828


#### What about removing only 10 the most popular words?

In [84]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)
max_review_length = 150
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

X_train = np.array([list(filter(lambda x: x > 10, review)) for review in X_train])
X_test = np.array([list(filter(lambda x: x > 10, review)) for review in X_test])

In [88]:
bimodel1 = BiModel(k1=1, n1=2, k2=0, n2=3)
bimodel1.fit(X_train, y_train)
print("Train data result:", accuracy_score(y_train, bimodel1.predict(X_train)))
print("Test data result:", accuracy_score(y_test, bimodel1.predict(X_test)))

Train data result: 0.99388
Test data result: 0.85984


In [89]:
bimodel2 = BiModel(k1=1, n1=2, k2=0, n2=2)
bimodel2.fit(X_train, y_train)
pred1 = bimodel2.predict(X_train)
pred2 = bimodel2.predict(X_test)
print("Train data result:", accuracy_score(y_train, pred1))
print("Test data result:", accuracy_score(y_test, pred2))

tn, fp, fn, tp = confusion_matrix(y_train, pred1).ravel()
print("For train:")
print("TN = {}, FP = {}, FN = {}, TP = {}".format(tn, fp, fn, tp))

tn, fp, fn, tp = confusion_matrix(y_test, pred2).ravel()
print("For test:")
print("TN = {}, FP = {}, FN = {}, TP = {}".format(tn, fp, fn, tp))

Train data result: 0.98656
Test data result: 0.85804
For train:
TN = 12214, FP = 286, FN = 50, TP = 12450
For test:
TN = 9670, FP = 2830, FN = 719, TP = 11781


For now, the best result for the training data is 0.99388 and for the test 0.85984.

In [90]:
# just to release memory
POOL.close()